In [1]:
import requests
import re


import pandas as pd
import numpy as np

In [2]:
genome_prefix = "https://rest.kegg.jp/get/gn:"

In [3]:
rx_entry = re.compile(r'(\d+)\s+(.+)\[TAX:(\d+)\]')

rx_gene = re.compile(r'\d+\s+(.+)')

In [4]:
with open("brite/brite_virus.txt", "r") as input:
    detail = input.readlines()

In [5]:
taxid_done = set()

with open("brite/taxon_protein.csv", "r") as input:
    for line in input.readlines():
        fields = line.strip().split(",")

        taxid_done.add(fields[0])

print (len(taxid_done))

taxid_gene = {}

virus_type = ""


for line in detail:
    line = line.strip()
        
    search_entry = rx_entry.search(line)
    #print (line)
    if search_entry:
        
        taxid = search_entry.group(1)

        if taxid not in taxid_done:

            url = genome_prefix + f"{taxid}"

            genome_detail = requests.get(url).text

            is_gene = False

            for line in genome_detail.split("\n"):
                if line.startswith("GENE"):
                    is_gene = True
                elif is_gene == True and not line.startswith(" "):
                    is_gene = False
                
                #print (is_gene)
                if is_gene == True:
                    genes = re.findall(rx_gene, line)

                    for g in genes:
                        with open("brite/taxon_protein.csv", "a") as output:
                            output.write(f"{taxid},{g}\n")

11487
